In [2]:
import os
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
from kafka import KafkaProducer
import json
from time import sleep
import nltk 
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to
[nltk_data]     /home/ciceromoura/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ciceromoura/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
#!wget https://repo1.maven.org/maven2/org/apache/spark/spark-streaming-kafka-0-8-assembly_2.11/2.4.5/spark-streaming-kafka-0-8-assembly_2.11-2.4.5.jar

In [4]:
conf = SparkConf().setAppName("Tweets").set("spark.jars", "./spark-streaming-kafka-0-8-assembly_2.11-2.4.5.jar")
sc = SparkContext(conf=conf)
ssc = StreamingContext(sc,1)

In [5]:
def send_to_kafka(word):
    word_topic = 'palavras-tweets'
    data = {'word': word[0], 'num': word[1]}
    producer = KafkaProducer(bootstrap_servers=['localhost:9092'])
    producer.send(word_topic, value=json.dumps(data).encode('utf-8'))
    producer.flush()
    return data

def remove_stopwords(word):
    stopwords_pt = stopwords.words('portuguese')
    stopwords_pt = stopwords_pt + ['RT', 'http', 'https']
    return word not in stopwords_pt

In [ ]:
tweets_topic = 'dados-tweets'
stream = KafkaUtils.createDirectStream(ssc, [tweets_topic], {"metadata.broker.list": 'localhost:9092'})
stream.count().map(lambda x:'Tweets nesse batch: %s' % x).pprint()
stream.map(lambda x: json.loads(x[1])['tweet']) \
      .flatMap(lambda x: x.split(" ")) \
      .countByValue()\
      .transform(lambda rdd:rdd.sortBy(lambda x:-x[1]))\
      .map(send_to_kafka)\
      .pprint()
ssc.start()
ssc.awaitTermination()

-------------------------------------------
Time: 2020-04-24 17:58:21
-------------------------------------------
Tweets nesse batch: 1

-------------------------------------------
Time: 2020-04-24 17:58:21
-------------------------------------------
{'word': 'da', 'num': 2}
{'word': 'RT', 'num': 1}
{'word': '@Emanoel1953:', 'num': 1}
{'word': 'Uma', 'num': 1}
{'word': 'singela', 'num': 1}
{'word': 'perguntinha...\n\nO', 'num': 1}
{'word': 'que', 'num': 1}
{'word': 'Sérgio', 'num': 1}
{'word': "'Banestado'", 'num': 1}
{'word': 'Moro,', 'num': 1}
...

-------------------------------------------
Time: 2020-04-24 17:58:22
-------------------------------------------
Tweets nesse batch: 1

-------------------------------------------
Time: 2020-04-24 17:58:22
-------------------------------------------
{'word': 'o', 'num': 3}
{'word': 'era', 'num': 2}
{'word': 'RT', 'num': 1}
{'word': '@Jacck13:', 'num': 1}
{'word': '#ForaBolsonaro\n\nE', 'num': 1}
{'word': 'no', 'num': 1}
{'word': 'fim', 'n